In [9]:
import pandas as pd
import util_functions as uf
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot, plot
from plotly.graph_objs import *
from plotly import tools
import plotly
import sys
sys.path.append("..")
from read_aws import *
plotly.offline.init_notebook_mode()

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [4]:
 # Connect to AWS
#uf.set_env_path()
#conn, cur = uf.aws_connect()
#Noah -Connect to AWS
conn = read_only_connect_aws()


# Trip Frequency Count for entire pilot by Operator
full_df = pd.read_sql("""select distinct
                    user_freqs.operatorclean,
                    user_freqs.user_trips,
                    count(*) as freq_user_trips
                    from
                    ((select distinct
                    operatorclean,
                    userid,
                    count(*) as user_trips
                    from dockless_trips
                    where operatorclean in ('lime', 'spin')
                    group by 1, 2
                    order by operatorclean, count(*))
                    union
                    /*ofo users*/
                    (select distinct
                    'ofo' as operatorclean,
                    userid,
                    sum(trips) as user_trips
                    from ofo_users
                    group by 1, 2
                    order by operatorclean, sum(trips))
                    union
                    /*jump users*/
                    (select distinct
                    'jump' as operatorclean,
                    userid,
                    sum(trips) as user_trips
                    from jump_users
                    group by 1, 2
                    order by operatorclean, sum(trips))) as user_freqs
                    group by 1, 2
                    order by 1, 2;
                 """, con=conn)
full_df.head()

,operatorclean,user_trips,freq_user_trips
0,jump,1,566
1,jump,2,354
2,jump,3,243
3,jump,4,185
4,jump,5,123


In [11]:
full_df.operatorclean.unique()

array(['jump', 'lime', 'ofo', 'spin'], dtype=object)

In [20]:
# Use jump Data as a test case
def get_data(operator, full_df):
    df = full_df[full_df['operatorclean'] == operator].copy()

    # Calculate Cumulative Sum and Perc
    df['cumulative_sum'] = df['freq_user_trips'].cumsum()
    df['cumulative_perc'] = (df['cumulative_sum'] / df['freq_user_trips'].sum()) * 100
    df['demarcation'] = 80
    print(df.head())
    return df


In [21]:
def traces(df): 
    trace1 = Bar(
        x=df.user_trips,
        y=df.freq_user_trips,
        name='Users by Frequency of Trips',
        marker=dict(
            color='rgb(34,163,192)'
                   )
    )
    trace2 = Scatter(
        x=df.user_trips,
        y=df.cumulative_perc,
        name='Cumulative Percentage',
        yaxis='y2',
        line=dict(
            color='rgb(243,158,115)',
            width=2.4
           )
    )
    trace3 = Scatter(
        x=df.user_trips,
        y=df.demarcation,
        name='80%',
        yaxis='y2',
        line=dict(
            color='rgba(128,128,128,.45)',
            dash = 'dash',
            width=1.5
           )
    )
    data = [trace1, trace2, trace3]
    return data

In [22]:
def lays(df, operator):
    layout = Layout(
        title='{0}'.format(operator),
        titlefont=dict(
            color='',
            family='',
            size=0
        ),
        font=Font(
            color='rgb(128,128,128)',
            family='Balto, sans-serif',
            size=12
        ),
        width=1500,
        height=623,
        paper_bgcolor='rgb(240, 240, 240)',
        plot_bgcolor='rgb(240, 240, 240)',
        hovermode='compare',
        margin=dict(b=250,l=60,r=60,t=65),
        showlegend=True,
           legend=dict(
              x=.83,
              y=1.3,
              font=dict(
                family='Balto, sans-serif',
                size=12,
                color='rgba(128,128,128,.75)'
            ),
        ),
        annotations=[ dict(
                      text="",
                      showarrow=False,
                      xref="paper", yref="paper",
                      textangle=90,
                      x=1.029, y=.75,
                      font=dict(
                      family='Balto, sans-serif',
                      size=14,
                      color='rgba(243,158,115,.9)'
                ),)],
        xaxis=dict(
          #tickangle=-90
        ),
        yaxis=dict(
            title='Users by Frequency of Trips',
            range=[0, max(df['freq_user_trips'])],
          tickfont=dict(
                color='rgba(34,163,192,.75)'
            ),
            titlefont=dict(
                    family='Balto, sans-serif',
                    size=14,
                    color='rgba(34,163,192,.75)')
        ),
        yaxis2=dict(
            range=[0,101],
            tickfont=dict(
                color='rgba(243,158,115,.9)'
            ),
            tickvals = [0,20,40,60,80,100],
            overlaying='y',
            side='right'
        )
    )
    return layout

In [25]:
def full_bunch(operator, main_df=full_df):
    df = get_data(operator, main_df)
    data = traces(df) 
    layout = lays(df,operator)
    axes = dict(data=data, layout=layout)
    return axes


In [30]:
fig = tools.make_subplots(rows=2, cols=2)
print (help(fig))
'''
tog = zip(full_df.operatorclean.unique(), )
for x in full_df.operatorclean.unique():
    ax = full_bunch(x)

fig.append_trace(trace1, 1, 1)
fig.append_trace(trace2, 1, 2)
fig.append_trace(trace2, 2, 1)
fig.append_trace(trace2, 2, 2)


iplot(fig)
plot(fig, auto_open=False, image = 'png', image_filename='plot_image', 
     output_type='file', image_width=800, image_height=600, 
     filename='full_test.html', validate=False)'''

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]
[ (2,1) x3,y3 ]  [ (2,2) x4,y4 ]

Help on Figure in module plotly.graph_objs.graph_objs object:

class Figure(PlotlyDict)
 |  Valid attributes for 'figure' at path [] under parents ():
 |  
 |      ['data', 'frames', 'layout']
 |  
 |  Run `<figure-object>.help('attribute')` on any of the above.
 |  '<figure-object>' is the object at []
 |  
 |  Method resolution order:
 |      Figure
 |      PlotlyDict
 |      builtins.dict
 |      PlotlyBase
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, *args, **kwargs)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  append_trace(self, trace, row, col)
 |      Add a trace to your figure bound to axes at the row, col index.
 |      
 |      The row, col index is generated from figures created with
 |      plotly.tools.make_subplots and can be viewed with
 |      Figure.print_grid.
 |      
 |      :param (dict) trace:

"\ntog = zip(full_df.operatorclean.unique(), )\nfor x in full_df.operatorclean.unique():\n    ax = full_bunch(x)\n\nfig.append_trace(trace1, 1, 1)\nfig.append_trace(trace2, 1, 2)\nfig.append_trace(trace2, 2, 1)\nfig.append_trace(trace2, 2, 2)\n\n\niplot(fig)\nplot(fig, auto_open=False, image = 'png', image_filename='plot_image', \n     output_type='file', image_width=800, image_height=600, \n     filename='full_test.html', validate=False)"